PREPARANDO EL ESPACIO

In [ ]:
import os
import time
import google.generativeai as genai #llama a la api de gemini
from tqdm.notebook import tqdm #para la barra de progreso

In [ ]:
# --- SETTINGS ---

#Configuración de Gemini
with open('gemini.txt', 'r') as f: # recuerda colocar tu clave de  API en el archivo gemini.txt
    my_api_key = f.read()
my_model = "gemini-2.5-flash" # selecciona tu modelo de preferencia, puedes ver los que estan disponibles aquí: https://ai.google.dev/gemini-api/docs/models
# Configura la API
genai.configure(api_key=my_api_key)
model = genai.GenerativeModel(my_model)

#Ruta: recuerda cambiar las rutas de las carpetas donde estarán tus store procedures y la ruta donde deseas que se carguen los archivos de documentación.
sp_location = "[UBICACIÓN_1]"
docutentation_output_location = "[UBICACIÓN_2]"

PREPARANDO FUNCIONES

In [ ]:
# --- FUNCIÓN PARA ENVIAR EL STORE PROCEDURE A GEMINI GENERAR LA DOCUMENTACIÓN ---
def generate_documentation(sql_code,file_name):
    """
    Envía el código SQL al LLM y retorna el bloque de comentarios.
    """

    prompt = f"""
    Rol: Eres un DBA Senior experto en T-SQL y procesos de consolidación financiera.
    Tarea: Generar documentación técnica estandarizada para el siguiente Store Procedure.
    
    Instrucciones de Formato:
    1. NO incluyas saludos, introducciones ni texto de relleno ("Aquí está tu análisis...").
    2. NO uses bloques de código (```markdown, ```text, etc). Devuelve texto plano directo.
    3. Sigue estrictamente la plantilla proporcionada abajo.
    4. Si un campo no aplica, escribe "N/A".
    
    Instrucciones de Análisis:
    - Categoria: Elige SOLAMENTE UNA de esta lista: [Consolidación de montos, Validación/Comparación de montos, Conversión local a dólar, Conversión dólar a sol, Otros].
    - Descripción: Enfócate en la lógica de negocio (el "por qué" y "para qué"), no describas línea por línea el código SQL.
    - Parámetros: Indica el nombre, tipo de dato y explica brevemente su uso. Si detectas que no se usa en el cuerpo del script, márcalo como "(NO UTILIZADO)".
    - Tablas: Separa claramente entre tablas que solo se leen (Source) y tablas que son modificadas (Target: INSERT, UPDATE, DELETE, MERGE).
    - Store Procedures ejecutados: Indica si, durante el proceso, se ejecuta otro/s store procedure/s.
    
    --- PLANTILLA DE INICIO ---
    Nombre Archivo: {file_name}
    Categoría: [CATEGORIA_AQUI]
    ----------------------------------------
    Descripción del Proceso:
    [RESUMEN_DE_NEGOCIO_AQUI]
    ----------------------------------------
    Parámetros:
    - @Parametro1 (Tipo): Explicación...
    - @Parametro2 (Tipo): Explicación... (NO UTILIZADO)
    ----------------------------------------
    Tablas Afectadas:
    [LECTURA]: TablaA, TablaB
    [ESCRITURA]: TablaC
    ----------------------------------------
    Store Procedures adicionales:
    - SP #1...
    - SP #2...
    --- PLANTILLA DE FIN ---
    
    Código SQL a analizar:
    {sql_code}
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"/* ERROR DURANTE LA DOCUMENTACIÓN: {str(e)} */"

In [ ]:
def procesar_archivos(list_of_files):
    print(f"Se encontraron {len(list_of_files)} archivos. Iniciando procesamiento...")

    # Usamos tqdm para ver una barra de progreso
    for name_of_file in tqdm(list_of_files):
        full_path = os.path.join(sp_location, name_of_file)
        
        with open(full_path, 'r') as f: #aseguraté de agregar el encoding correcto si tus archivos tienen caracteres especiales. Ejemplo: puedes usar encoding='utf-8'
            codigo_sp = f.read()

        # Generar docs
        doc_header = generate_documentation(codigo_sp,name_of_file)

        #ruta archivo output
        path_new_file = docutentation_output_location+"DOCU__"+name_of_file+".txt"
        
        with open(path_new_file, "w", encoding='utf-8') as new_file:
            new_file.write(doc_header)
        
        # IMPORTANTE: Pausa para evitar problemas con el Rate Limit
        time.sleep(5)

PROCESAR LOS ARCHIVOS

In [ ]:
# Recoge la lista de archivos SQL en la carpeta especificada
l_files = [f for f in os.listdir(sp_location) if f.endswith('.sql')]

# Si deseas probar con un número menor de archivos, puedes descomentar la siguiente línea para seleccionar solo un subconjunto de los archivos (ejemplo: los primeros 10 archivos):
# archivos = archivos[:10]

#Muestra la lista de archivos que se van a procesar
l_files

In [ ]:
#Muestra el número de archivos que se van a procesar
print("Number of files to process:", len(l_files))

In [ ]:
#Finalmente, llama a la función para procesar los archivos
procesar_archivos(l_files)

REVISA POSIBLES ERRORES EN LOS ARCHIVOS DE LOS STORE PROCEDURE

In [ ]:
#Utiliza este código para verificar posibles errores al leer los archivos, por ejemplo, debido a problemas de encoding (codificación).
#Intentará leer todos los archivos y devolverá una lista de los archivos que no se pudieron leer.
errors = []

for file_name in tqdm(l_files):
    full_path = os.path.join(sp_location, file_name)

    try:
        with open(full_path, 'r') as f: #agrega el encoding que deseas probar, por ejemplo, encoding='utf-16'
            a = f.read()
    except Exception as e:
        errors.append(file_name)

print("Número de archivos con errores: "+str(len(errors)))
print(errors)